In [1]:
!pip install ijson
!pip install namu-wiki-extractor

     |████████████████████████████████| 133kB 29.6MB/s 
  Created wheel for namu-wiki-extractor: filename=namu_wiki_extractor-0.2.1-cp37-none-any.whl size=3806 sha256=ec18c351ea69f01e765215522dccddc07e02af7a602c0acda6e3611fb919e593
  Stored in directory: /root/.cache/pip/wheels/23/51/bc/736f02777e347914c066b7d5bde26190aed588354b9503f3c3
Successfully built namu-wiki-extractor


In [2]:
import re
import ijson
from namuwiki.extractor import extract_text
import json

word_lst = ['kbo','baseball','베이스볼','야구',]

capture_values = [("item.namespace", "string"),
                  ("item.title", "string"),
                  ("item.text", "string")]

def parse_namuwiki_json():
    i = 0
    doc = {}
    with open('/content/drive/MyDrive/Colab Notebooks/namuwiki_20210301.json') as f:
        for prefix, event, value in ijson.parse(f):
            if (prefix, event) in capture_values:
                doc[prefix[5:]] = value
            if (prefix, event, value) == ("item", "end_map", None):
                yield doc
                doc = {}
                i += 1

cleaning_first_patterns = [r"\[\*[^\]]+\]",r"~~[^~]+~~"]
cleaning_first_patterns = [re.compile(pattern, re.IGNORECASE | re.MULTILINE) for pattern in cleaning_first_patterns]

cleaning_patterns = [r"\([^\)]+\)"]
cleaning_patterns = [re.compile(pattern, re.IGNORECASE | re.MULTILINE) for pattern in cleaning_patterns]

replace_patterns = {'\\n': "\n", "\\'": "'"}

def clean_text(text):
    for regex in cleaning_first_patterns:
        text = re.sub(regex, "", text)
    text = extract_text(text)
    for regex in cleaning_patterns:
        text = re.sub(regex, "", text)
    for k, v in replace_patterns:
        text = text.replace(k, v)
    return text

document = {}

for i, doc in enumerate(parse_namuwiki_json()):
    title = doc['title']
    text = clean_text(doc['text'])
    for word in word_lst:
        if text.lower().count(word) >= 4:
            document[title] = text

# 2 | 16272
# 3 | 11591
# 4 | 9055

print(len(list(document.items())))

9055


In [3]:
import zipfile
         
f = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/club.zip')
f.extractall('/content/club')
f.close()

f = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/player.zip')
f.extractall('/content/player')
f.close()

In [4]:
import os

for file_name in os.listdir('/content/club/club'):
    if file_name != '.DS_Store':
        with open('/content/club/club/'+file_name, 'r') as f:
            contents = f.read()
        document[file_name.split('.')[0]] = contents

In [5]:
import os

for file_name in os.listdir('/content/player/player'):
    if file_name != '.DS_Store':
        with open('/content/player/player/'+file_name, 'r') as f:
            contents = f.read()
        document[file_name.split('.')[0]] = contents

In [6]:
print(len(list(document.items())))

with open('namuwiki_baseball.json', 'w') as f:
    json.dump(document, f, ensure_ascii = False)

11230
